In [1]:
%pylab notebook
import numpy as np
import pandas as pd
import os
from datetime import datetime
from cognite import CogniteClient
from cognite.client.stable.datapoints import TimeseriesWithDatapoints, Datapoint
import csv

# sam-jetfire
client = CogniteClient(api_key='API-KEY')

Populating the interactive namespace from numpy and matplotlib


In [4]:
# List of datapoints being processed 
current_dp = []

# Maximum number of datapoints batched at once
batch_max = 100

# TO DO: LOOP THROUGH ALL CSVs STORED IN A GIVEN LOCATION 

# Read CSV
df = pd.read_csv('../TebisSampleData/TEBIS_FK_1546297260.csv', encoding='latin-1', delimiter=';', quotechar='"', index_col=0)

# Remove first row (unit of data)
df = df[1:]

for index, row in df.iterrows():
    for col in df:
        
        # Filter out null values
        if pd.notnull(row[col]):
            
            # Post datapoints if list of datapoints being processed is greater than the maximum, then empty list
            if len(current_dp) >= batch_max:
                client.datapoints.post_multi_time_series_datapoints(current_dp)
                current_dp = []
            
            # Format Datapoint and TimeseriesWithDatapoints
            ts = str(col.rpartition(':')[2].strip())
            dp = Datapoint(timestamp=int(index), value=float(row[col].replace(',', '.')))
            current_dp.append(TimeseriesWithDatapoints(name=ts, datapoints=[dp]))

# If list of datapoints being processed is not empty, post leftover datapoints
if current_dp:
    client.datapoints.post_multi_time_series_datapoints(current_dp)
    current_dp = []